d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 1200px">
</div>

-sandbox
# Model Interpretability

<img style="width:20%" src="https://files.training.databricks.com/images/Limes.jpg" > No, we're not talking about limes.

We're talking about [Local Interpretable Model-Agnostic Explanations](https://github.com/marcotcr/lime).

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Use LIME and SHAP to understand which features are most important in the model's prediction for that data point

In [3]:
%pip install shap==0.35.0

In [4]:
%pip install lime==0.2.0.0

In [5]:
%run "./Includes/Classroom-Setup"

Don't forget you need to scale your data because the model was trained on scaled data!

In [7]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

boston_housing = load_boston() 

# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(boston_housing.data,
                                                    boston_housing.target,
                                                    test_size=0.2,
                                                    random_state=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(boston_housing.DESCR)

## Load in Saved Model

In [9]:
from tensorflow.keras.models import load_model

filepath = "/dbfs/mnt/training/bostonhousing/model.hdf5"

model = load_model(filepath)

### Using LIME for Model Explanation
We can use the [LIME](https://github.com/marcotcr/lime) library to provide explanations of individual predictions.

![](https://raw.githubusercontent.com/marcotcr/lime/master/doc/images/lime.png)

In [11]:
import lime
from lime.lime_tabular import LimeTabularExplainer

help(LimeTabularExplainer)

###Interpreting Results
LIME explains the impact of each feature on the prediction, and tabular explainers need a training set. 

The reason for this is because we compute statistics on each feature (column). If the feature is numerical, we compute the mean and std, and discretize it into quartiles.

In [13]:
def model_predict(input):
  """
  Convert keras prediction output to LIME compatible form. (Needs to output a 1 dimensional array)
  """
  return model.predict(input).flatten()

explainer = LimeTabularExplainer(X_train, feature_names=boston_housing.feature_names, class_names=["price"], mode="regression")
# NOTE: In order to pass in categorical_features, they all need to be ints

From the LIME docs:

0. First, we generate neighborhood data by randomly hiding features from the instance. 
0. We then learn locally weighted linear models on this neighborhood data to explain each of the classes in an interpretable way.

In [15]:
exp = explainer.explain_instance(X_test[0], model_predict, num_features=10)
print(f"True value: {y_test[0]}")
print(f"Local predicted value: {exp.predicted_value}")

## Which features are most important?

In [17]:
displayHTML(exp.as_html())

Positive correlation: 
0. `RM` (average number of rooms per dwelling) 
0. `CRIM` (per capita crime rate by town) - below "average"
0. `LSTAT` (% lower status of the population)
0. `B`     (1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town)
0. `TAX`   (full-value property-tax rate per $10,000)
0. `INDUS` (proportion of non-retail business acres per town)

Negative correlation:
0. `CHAS` (Charles River dummy variable = 1 if tract bounds river; 0 otherwise)
0. `ZN` (proportion of residential land zoned for lots over 25,000 sq.ft.)
0. `RAD` (index of accessibility to radial highways)
0. `PTRATIO`  (pupil-teacher ratio by town)

Do these make sense?

In [19]:
display(exp.as_pyplot_figure().tight_layout())

Let's get those values as a list.

In [21]:
exp.as_list()

## SHAP

SHAP ([SHapley Additive exPlanations](https://github.com/slundberg/shap)) is another approach to explain the output of a machine learning model. See the [SHAP NIPS](http://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions) paper for details, and Christoph Molnar's book chapter on [Shapley Values](https://christophm.github.io/interpretable-ml-book/shapley.html).

![](https://raw.githubusercontent.com/slundberg/shap/master/docs/artwork/shap_diagram.png)

Great [blog post](https://blog.dominodatalab.com/shap-lime-python-libraries-part-1-great-explainers-pros-cons/) comparing LIME to SHAP. SHAP provides greater theoretical guarantees than LIME, but at the cost of additional compute.

In [23]:
import shap

help(shap.DeepExplainer)

In [24]:
# from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import tensorflow as tf

shap.initjs()
shap_explainer = shap.DeepExplainer(model, X_train[:200])
shap_values = shap_explainer.shap_values(X_test[0:1])
y_pred = model.predict(X_test[0:1])
print(f"Actual price: {y_test[0]}, Predicted price: {y_pred[0][0]}")
                   
# Saving to File b/c can't display IFrame directly in Databricks: https://github.com/slundberg/shap/issues/101
file_path = "shap.html"
shap.save_html(file_path, shap.force_plot(shap_explainer.expected_value[0].numpy(), 
                                          shap_values[0], 
                                          X_test[0:1],
                                          feature_names=boston_housing.feature_names, 
                                          show=False))

## Visualize

* Red pixels increase the model's output while blue pixels decrease the output.

Here's a great [article](https://christophm.github.io/interpretable-ml-book/shapley.html) discussing how SHAP works under the hood.

In [26]:
import codecs

f = codecs.open(file_path, "r")
displayHTML(f.read())

The values on the bottom show the true values of `X_test[0]`.

In [28]:
import pandas as pd

pd.DataFrame(X_test[0], boston_housing.feature_names, ["features"])

This says that, overall, factors like negative LSTAT had the most positive effect on predicted housing price, while ZN had the most negative, among others.
```
- ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
- AGE      proportion of owner-occupied units built prior to 1940
- PTRATIO  pupil-teacher ratio by town
- RAD      index of accessibility to radial highways
- CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)

- LSTAT    % lower status of the population
- RM       average number of rooms per dwelling
- INDUS    proportion of non-retail business acres per town
- TAX      full-value property-tax rate per $10,000
- B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- CRIM     per capita crime rate by town
- NOX      nitric oxides concentration (parts per 10 million)
- DIS      weighted distances to five Boston employment centres
```

Let's see what the values corresponding to each feature are.

In [31]:
shap_features = pd.DataFrame(shap_values[0][0], boston_housing.feature_names, ["features"])
shap_features.sort_values("features")

The predicted value is the sum of the SHAP features + the average.

In [33]:
print(f"The predicted value is {shap_features.sum()[0] + shap_explainer.expected_value[0]}")

**Question**: How similar are the LIME predictions to the SHAP predictions?

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>